In [1]:
# !git clone https://github.com/cg123/mergekit.git
# %cd mergekit
# %pip install -e .

In [2]:
# ! export TRANSFORMERS_CACHE="/dtu/blackhole/00/167776/cache"

In [8]:
import os

In [9]:
os.chdir('/dtu/blackhole/06/187238/mergekit')

In [10]:
print("Current working directory:", os.getcwd())

Current working directory: /dtu/blackhole/06/187238/mergekit


In [11]:
OUTPUT_PATH = "./merged"  # folder to store the result in
LORA_MERGE_CACHE = "./merge_cache"  # change if you want to keep these for some reason
CONFIG_YML = "./examples/meta-bio.yml"  # merge configuration file
COPY_TOKENIZER = True  # you want a tokenizer? yeah, that's what i thought
LAZY_UNPICKLE = False  # experimental low-memory model loader
LOW_CPU_MEMORY = False  # enable if you somehow have more VRAM than RAM+swap

In [12]:
# actually do merge
import torch
import yaml

from mergekit.config import MergeConfiguration
from mergekit.merge import MergeOptions, run_merge

with open(CONFIG_YML, "r", encoding="utf-8") as fp:
    merge_config = MergeConfiguration.model_validate(yaml.safe_load(fp))

run_merge(
    merge_config,
    out_path=OUTPUT_PATH,
    options=MergeOptions(
        lora_merge_cache=LORA_MERGE_CACHE,
        cuda=torch.cuda.is_available(),
        copy_tokenizer=COPY_TOKENIZER,
        lazy_unpickle=LAZY_UNPICKLE,
        low_cpu_memory=LOW_CPU_MEMORY,
    ),
)
print("Done!")

Warmup loader cache:   0%|                                                                                                         | 0/3 [00:00<?, ?it/s]

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

Warmup loader cache:  67%|████████████████████████████████████████████████████████████████▋                                | 2/3 [00:00<00:00,  4.03it/s]


GatedRepoError: 401 Client Error. (Request ID: Root=1-675020ce-08c4d84b530816bc2d7e6c10;9061745f-d033-4244-819e-ed60524b3231)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1/resolve/2dcff66eac0c01dc50e4c41eea959968232187fe/pytorch_model.bin.index.json.
Access to model mistralai/Mistral-7B-Instruct-v0.1 is restricted. You must have access to it and be authenticated to access it. Please log in.

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from datasets import Dataset, load_dataset
import json
import sys


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("merged", cache_dir='./', device_map="cuda:0")
model = AutoModelForCausalLM.from_pretrained("merged", cache_dir='./', torch_dtype=torch.float16, device_map="cuda:0")

TypeError: __init__() got an unexpected keyword argument 'response_format'

In [28]:
# Define the prompt
prompt = """
Please answer the following question strictly in JSON format. Your response must contain only a JSON object 
with the keys "Explanation" and "Answer_choice". The "Answer_choice" should be one of three options: "A", "B", or "C". 
No extra text or explanation outside the JSON format is allowed.

Question: if x^2=9, then what is x?
A: 1
B: 2
C: 3
D: 5
"""
sent = tokenizer([prompt], return_tensors="pt").to("cuda:0")
output = model.generate(**sent, max_new_tokens=100, do_sample=True, pad_token_id=tokenizer.eos_token_id).to("cuda:0")
print(tokenizer.batch_decode(output)[0])

<s> 
Please answer the following question strictly in JSON format. Your response must contain only a JSON object 
with the keys "Explanation" and "Answer_choice". The "Answer_choice" should be one of three options: "A", "B", or "C". 
No extra text or explanation outside the JSON format is allowed.

Question: if x^2=9, then what is x?
A: 1
B: 2
C: 3
D: 5
E: 6

{ "Explanation": "Since x^2 = 9, we can find the value of x by taking the square root of both sides. 9 is a perfect square, so its square root is easily found to be 3. Therefore, the value of x is 3.", "Answer_choice": "C" }</s>


In [25]:
from huggingface_hub import InferenceClient
client = InferenceClient("merged", cache_dir='./', device_map="cuda:0")

TypeError: __init__() got an unexpected keyword argument 'cache_dir'